In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 49.4 MB/s eta 0:00:00


In [2]:

import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt

In [3]:
def hydrogen_hamiltonian(coordinates):
    """Calculates the qubit Hamiltonian of the hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (qml.Hamiltonian): A PennyLane Hamiltonian.
    """
    return qml.qchem.molecular_hamiltonian(["H", "Li"], coordinates, charge=0, basis="STO-3G",active_electrons=2)[0]


def hf(electrons, num_qubits):
    """Calculates the Hartree-Fock state of the hydrogen molecule.

    Args:
        electrons (int): The number of electrons in the hydrogen molecule.
        num_qubits (int): The number of qubits needed to represent the hydrogen molecule Hamiltonian.

    Returns:
        (numpy.tensor): The HF state.
    """
    # Put your solution here #
    return qml.qchem.hf_state(electrons=electrons, orbitals=num_qubits)

Using exact diagonalization to get the ground state and first excited state.

In [5]:
L=1.5
coordinates = np.array([0.0, 0.0, -L/2, 0.0, 0.0, L/2]) #np.array([-L/2/np.sqrt(3), L/2, 0.0, L/np.sqrt(3), 0.0, 0.0,-L/2/np.sqrt(3), -L/2, 0.0])
charge=0
hamiltonian = hydrogen_hamiltonian(np.array(coordinates))
H_matrix=qml.matrix(hamiltonian)

In [6]:


vals, vecs = np.linalg.eigh(H_matrix)

inds=np.argsort(vals)
eng=vals[inds]
vec=vecs[:,inds]

print('The ground state energy:',eng[0])


The ground state energy: -7.626624465547493


In [7]:
def vec_check(vec,electron_num):
    for i,e in enumerate(vec):
        if np.abs(e) > 1e-8:
            e_occ=bin(i)[2:]
            lst=' '.join(e_occ).split(' ')
            cv=tuple(map(int, lst))
            e_num=sum(cv)
            if e_num!=electron_num:
                return False
    return True

In [9]:
sz=qml.qchem.spinz(len(hamiltonian.wires))
SZ=qml.matrix(sz)
Stop=[False,False,False]
elec_num=2
for i in range(1,len(eng)): # Finding first excited state with -1 Sz
    Sz=vec[:,i].dot(SZ.dot(np.transpose(np.conjugate(vec[:,i]))))

    if np.abs(Sz) < 1e-5 and vec_check(vec[:,i],elec_num):
        print('The first excied state energy with sz=0:',eng[i])
        Stop[0]=True

    elif np.abs(Sz-1) < 1e-5 and vec_check(vec[:,i],elec_num):
        print('The first excied state energy with sz=1:',eng[i])
        Stop[1]=True

    elif np.abs(Sz+1) < 1e-5 and vec_check(vec[:,i],elec_num):
        print('The first excied state energy with sz=-1:',eng[i])
        Stop[2]=True

    if np.all(Stop):
        break

The first excied state energy with sz=1: -7.522914627103037
The first excied state energy with sz=0: -7.522914627103034
The first excied state energy with sz=-1: -7.52291462710303


In [20]:
def bit_to_vec(bitstring_dict,num_qubits):
    vec=np.zeros(2**num_qubits)
    amp=0
    for i in range(2**num_qubits):
        bitstr=bin(i)[2:]
        if len(bitstr) < num_qubits:
            bitstr=(num_qubits-len(bitstr))*'0'+bitstr

        if bitstr in bitstring_dict:
            vec[i]=bitstring_dict[bitstr]
            amp+=np.abs(bitstring_dict[bitstr])**2
    vec=vec/np.sqrt(amp)
    return vec

In [36]:
class SS_vqe_ch():
    def __init__(self,state_lst,num_electrons,symbol,geometry,charge) -> None:
        self.num_electrons=num_electrons
        self.symbol=symbol
        self.geometry=geometry
        self.charge=charge

        self.hamiltonian=self.Ch_hamiltonian(symbol,geometry,num_electrons,charge)
        self.num_qubits = len(self.hamiltonian.wires)
        self.sz=qml.qchem.spinz(self.num_qubits)
        self.singles, self.doubles = qml.qchem.excitations(self.num_electrons, self.num_qubits)
        self.state_lst=[]
        for stat in state_lst:
            self.state_lst.append(bit_to_vec(stat,self.num_qubits))
        H_matrix=qml.matrix(self.hamiltonian)
        H_sq=H_matrix.dot(H_matrix)

        self.H_sq_ob=qml.Hermitian(H_sq, wires=range(self.num_qubits))
        self.dev = qml.device("default.qubit", wires=self.num_qubits)
        self.cost_h_fn = qml.QNode(self.circuit, self.dev)
        self.Sz_fn = qml.QNode(self.circuit_sz, self.dev)

    def Ch_hamiltonian(self,symbol,geometry,num_electrons,charge):
        return qml.qchem.molecular_hamiltonian(symbol, geometry, charge=charge,\
                                               basis="STO-3G",active_electrons=num_electrons)[0]

    def hf(self,electrons, num_qubits):
        return qml.qchem.hf_state(electrons=electrons, orbitals=num_qubits)

    def circuit(self,state,weights):
        qml.StatePrep(state, wires=range(self.num_qubits))
        d_l=len(self.doubles)
        s_l=len(self.singles)
        for j in range(self.num_layer):
            for i in range(len(self.singles)):
                qml.SingleExcitation(weights[i+d_l+j*(d_l+s_l)], wires=self.singles[i])
            for i in range(len(self.doubles)):
                qml.DoubleExcitation(weights[i+j*(d_l+s_l)], wires=self.doubles[i])
        return qml.expval(self.hamiltonian)


    def circuit_sz(self,state,weights):
        qml.StatePrep(state, wires=range(self.num_qubits))
        d_l=len(self.doubles)
        s_l=len(self.singles)
        for j in range(self.num_layer):
            for i in range(len(self.singles)):
                qml.SingleExcitation(weights[i+d_l+j*(d_l+s_l)], wires=self.singles[i])
            for i in range(len(self.doubles)):
                qml.DoubleExcitation(weights[i+j*(d_l+s_l)], wires=self.doubles[i])
        return qml.expval(self.sz)

    def cost_fn(self,params):
        cost=0
        for ind, stat in enumerate(self.state_lst):
            cost+=self.w[ind]*self.cost_h_fn(stat,params)
        return cost

    def run(self,w=None,epochs=400,stepsize=0.5,num_layer=2,cov=1e-6,print_train=True):
        if w==None:
            self.w=[1.0]+(len(self.state_lst)-1)*[0.5]
        else:
            self.w=w

        self.num_layer=num_layer
        opt = qml.AdamOptimizer(stepsize=stepsize)
        self.weights =np.zeros(num_layer*len(self.doubles + self.singles), requires_grad=True)

        i=0
        iter=[]
        cost_val=[]
        Lowest_cost=0
        self.best_weights=self.weights

        cov=cov
        E_pev=[0.0]*len(self.state_lst)
        E_lst=[0.0]*len(self.state_lst)
        E_pev=np.array(E_pev)
        E_lst=np.array(E_lst)
        for _ in range(epochs):
            self.weights = opt.step(self.cost_fn, self.weights)
            iter.append(i)
            cost=self.cost_fn(self.weights)
            for j,stat in enumerate(self.state_lst):
                E_lst[j]=self.cost_h_fn(stat,self.weights)

            if Lowest_cost>cost:
                Lowest_cost=cost
                self.best_weights=self.weights
            cost_val.append(E_lst)

            i=i+1
            if print_train:
                print('iter:',i)
                for j,stat in enumerate(self.state_lst):
                    print(' E0:',E_lst[j],' Sz:',self.Sz_fn(stat,self.weights))
            E_diff=np.abs(E_lst-E_pev)
            if np.all(E_diff<cov):
                break

            #for k in range(len(self.state_lst)):
            E_pev=E_lst.copy()
        self.E_lst=E_lst
        return E_lst

In [37]:
L=1.5
symbol=["H", "Li"]
geometry=np.array([0.0, 0.0, -L/2, 0.0, 0.0, L/2])

charge=0
num_electrons=2
g_stat={'1100000000': 1}
e_stat_p1={'1010000000': 1}
e_stat_m1={'0101000000': 1}
e_stat_0={'0110000000': 1,'1001000000': 1}
stat_lst=[g_stat,e_stat_p1,e_stat_m1,e_stat_0]
ss_vqe=SS_vqe_ch(stat_lst,num_electrons,symbol,geometry,charge)

In [39]:
E0,E1p1,E1m1,E1_0=ss_vqe.run()

iter: 1
 E0: -7.114361086861115  Sz: 0.0
 E0: -7.464140817197321  Sz: 1.0
 E0: -7.464032961677203  Sz: -1.0
 E0: -7.4639406502334085  Sz: -8.326672684688674e-17
iter: 2
 E0: -7.206854536072497  Sz: 0.0
 E0: -7.49408793966439  Sz: 1.0
 E0: -7.498316375839986  Sz: -1.0
 E0: -7.495944851828021  Sz: 5.551115123125783e-17
iter: 3
 E0: -7.403765724131152  Sz: -5.551115123125783e-17
 E0: -7.510131345584048  Sz: 1.0
 E0: -7.513275162000521  Sz: -1.0
 E0: -7.500083460776256  Sz: 0.0
iter: 4
 E0: -7.388160843037501  Sz: 5.551115123125783e-17
 E0: -7.515238978025504  Sz: 0.9999999999999997
 E0: -7.516227935536564  Sz: -0.9999999999999997
 E0: -7.510460289070379  Sz: -2.7755575615628914e-17
iter: 5
 E0: -7.473409277364713  Sz: 2.7755575615628914e-17
 E0: -7.49417707357977  Sz: 0.9999999999999999
 E0: -7.50596459664734  Sz: -1.0
 E0: -7.500834585541014  Sz: 0.0
iter: 6
 E0: -7.463527924576138  Sz: 2.7755575615628914e-17
 E0: -7.493139935867371  Sz: 1.0
 E0: -7.509698517436156  Sz: -0.99999999999999

In [10]:
electrons = 2
num_qubits = len(hamiltonian.wires)

S2 = qml.qchem.spin2(electrons, num_qubits)
sz=qml.qchem.spinz(num_qubits)

singles, doubles = qml.qchem.excitations(electrons, num_qubits)
pool_operators=singles+doubles

num_qubits = len(hamiltonian.wires)
hf_state = hf(electrons, num_qubits)

dev = qml.device("default.qubit", wires=num_qubits)
print("The original vqe use ",len(doubles)*13 + len(singles)*2," cnot gates")
print("The original vqe use ",len(doubles)*34 + len(singles)*10," gates")

bitstr=''.join(map(str, hf_state))

g_trial_stat={bitstr: 1}
e_trial_stat_szp1={'1010000000': 1}
e_trial_stat_szm1={'0101000000': 1}
e_trial_stat_sz0={'1001000000': 1,'0110000000': 1}

def bit_to_vec(bitstring_dict,num_qubits):
    vec=np.zeros(2**num_qubits)
    amp=0
    for i in range(2**num_qubits):
        bitstr=bin(i)[2:]
        if len(bitstr) < num_qubits:
            bitstr=(num_qubits-len(bitstr))*'0'+bitstr
        #print(bitstr)
        if bitstr in bitstring_dict:
            vec[i]=bitstring_dict[bitstr]
            amp+=np.abs(bitstring_dict[bitstr])**2
    vec=vec/np.sqrt(amp)
    return vec

g_stat=bit_to_vec(g_trial_stat,num_qubits)
e_stat_szp1=bit_to_vec(e_trial_stat_szp1,num_qubits)
e_stat_szm1=bit_to_vec(e_trial_stat_szm1,num_qubits)
e_stat_sz0=bit_to_vec(e_trial_stat_sz0,num_qubits)

def circuit(state,weights):
    qml.StatePrep(state, wires=range(num_qubits))
    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i], wires=doubles[i])

    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+2*len(doubles)+len(singles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i+len(doubles)+len(singles)], wires=doubles[i])

    return qml.expval(hamiltonian)

def circuit_sz(state,weights):
    qml.StatePrep(state, wires=range(num_qubits))
    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i], wires=doubles[i])

    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+2*len(doubles)+len(singles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i+len(doubles)+len(singles)], wires=doubles[i])
    return qml.expval(sz)


cost_h_fn = qml.QNode(circuit, dev)
Sz_fn = qml.QNode(circuit_sz, dev)

def cost_fn(params):
    return cost_h_fn(g_stat,params)+0.5*cost_h_fn(e_stat_szp1,params)+0.5*cost_h_fn(e_stat_szm1,params)\
            +0.5*cost_h_fn(e_stat_sz0,params)

weights =np.zeros(2*len(doubles + singles), requires_grad=True)

opt = qml.AdamOptimizer(stepsize=0.5)

i=0
iter=[]
cost_val=[]
Lowest_E=0
best_weights=0

cov=1e-5
E0_pev=0
E1_pev=0
E2_pev=0
E3_pev=0

for _ in range(400):
    weights = opt.step(cost_fn, weights)
    iter.append(i)

    E0=cost_h_fn(g_stat,weights)
    E1=cost_h_fn(e_stat_szp1,weights)
    E2=cost_h_fn(e_stat_sz0,weights)
    E3=cost_h_fn(e_stat_szm1,weights)

    if Lowest_E>E0:
        Lowest_E=E0
        best_weights=weights
    cost_val.append(E0)

    i=i+1
    print('Need iter:',i,' E0:',E0,', E1:',E1,' Sz:',Sz_fn(e_stat_szp1,weights))
    print('E2:',E2,Sz_fn(e_stat_sz0,weights))
    print('E3:',E3,Sz_fn(e_stat_szm1,weights))
    if np.abs(E0-E0_pev)<cov and np.abs(E1-E1_pev)<cov and np.abs(E2-E2_pev)<cov and np.abs(E3-E3_pev)<cov:
        print('Need iter:',i,' E0:',E0,', E1:',E1,' Sz:',Sz_fn(e_stat_szp1,weights))
        print('E2:',E2,Sz_fn(e_stat_sz0,weights))
        print('E3:',E3,Sz_fn(e_stat_szm1,weights))
        break

    E0_pev=E0
    E1_pev=E1
    E2_pev=E2
    E3_pev=E3

The original vqe use  224  cnot gates
The original vqe use  624  gates
Need iter: 1  E0: -7.114361086861115 , E1: -7.464140817197321  Sz: 1.0
E2: -7.4639406502334085 -8.326672684688674e-17
E3: -7.464032961677203 -1.0
Need iter: 2  E0: -7.206854536072497 , E1: -7.49408793966439  Sz: 1.0
E2: -7.495944851828021 5.551115123125783e-17
E3: -7.498316375839986 -1.0
Need iter: 3  E0: -7.403765724131152 , E1: -7.510131345584048  Sz: 1.0
E2: -7.500083460776256 0.0
E3: -7.513275162000521 -1.0
Need iter: 4  E0: -7.388160843037501 , E1: -7.515238978025504  Sz: 0.9999999999999997
E2: -7.510460289070379 -2.7755575615628914e-17
E3: -7.516227935536564 -0.9999999999999997
Need iter: 5  E0: -7.473409277364713 , E1: -7.49417707357977  Sz: 0.9999999999999999
E2: -7.500834585541014 0.0
E3: -7.50596459664734 -1.0
Need iter: 6  E0: -7.463527924576138 , E1: -7.493139935867371  Sz: 1.0
E2: -7.496506836570258 0.0
E3: -7.509698517436156 -0.9999999999999998
Need iter: 7  E0: -7.402949539510246 , E1: -7.507295496629

In [19]:

electrons = 2
num_qubits = len(hamiltonian.wires)

singles, doubles = qml.qchem.excitations(electrons, num_qubits)
pool_operators=singles+doubles

num_qubits = len(hamiltonian.wires)
hf_state = hf(electrons, num_qubits)

def circuit_1(state,weights,excitations):
    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(weights[i], wires=excitation)
        else:
            qml.SingleExcitation(weights[i], wires=excitation)
    return qml.expval(hamiltonian)

def circuit_sz(state,weights,excitations):
    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(weights[i], wires=excitation)
        else:
            qml.SingleExcitation(weights[i], wires=excitation)
    return qml.expval(sz)

dev = qml.device("default.qubit", wires=num_qubits)#

cost_h_fn = qml.QNode(circuit_1, dev, expansion_strategy="device")
Sz_fn = qml.QNode(circuit_sz, dev)

def cost_fn(params, excitations):
    return cost_h_fn(g_stat,params, excitations)+0.45*cost_h_fn(e_stat_szp1,params, excitations)\
            +0.45*cost_h_fn(e_stat_szm1,params, excitations)+0.2*cost_h_fn(e_stat_sz0,params, excitations)


epochs=400

energy=[]
operator_circuits=[]# doubles_select+singles_select

weights = [] #[0.0] * len(operator_circuits)
weights=np.array(weights)

opt = opt = qml.GradientDescentOptimizer(stepsize=0.4)

circuit_gradient = qml.grad(cost_fn, argnum=0)


E0 = cost_h_fn(g_stat,weights, operator_circuits)
E1 = cost_h_fn(e_stat_szp1,weights, operator_circuits)
E2 = cost_h_fn(e_stat_sz0,weights, operator_circuits)
E3 = cost_h_fn(e_stat_szm1,weights, operator_circuits)

print(f"Epoch = 0, G Energy = {E0:.8f} Ha, E1 Energy = {E1:.8f} Ha,")
print(f" E2 Energy = {E1:.8f} Ha")
print("Number of gates = {}\n".format(len(operator_circuits)))

configs=qml.specs(cost_h_fn)(g_stat,weights, excitations=operator_circuits)

#print(configs)
print("Circuit depth {}\n".format(configs['resources'].depth))

conv=1e-6
E0_pev=0
E1_pev=0
E2_pev=0
E3_pev=0
for n in range(epochs):

    Test_set=operator_circuits+pool_operators
    Test_params=list(weights)+[0.0]*len(pool_operators)
    Test_params=np.array(Test_params)


    test_grads = circuit_gradient(Test_params,Test_set)
    test_grads = test_grads[len(weights):]
    maxpos = np.argmax(np.abs(test_grads))
    print('Max grad',np.max(np.abs(test_grads)))

    if n<= 40 or np.abs(test_grads[maxpos])<0.05:
        max_op=pool_operators[maxpos]
        operator_circuits.append(max_op)
        weights=np.append(weights, 0.0)

    weights,_ = opt.step(cost_fn, weights, operator_circuits) # Step 11.
    E0 = cost_h_fn(g_stat,weights,operator_circuits)
    E1 = cost_h_fn(e_stat_szp1,weights, operator_circuits)
    E2 = cost_h_fn(e_stat_sz0,weights, operator_circuits)
    E3 = cost_h_fn(e_stat_szm1,weights, operator_circuits)

    print(f"Epoch = {n+1}, G Energy = {E0:.8f} Ha, Sz = {Sz_fn(g_stat,weights,operator_circuits):.8f} Ha,")
    print(f" The first excited state Energy = {E1:.8f} Ha, Sz= {Sz_fn(e_stat_szp1,weights,operator_circuits):.8f}")
    print(f" The first excited state Energy = {E2:.8f} Ha, Sz= {Sz_fn(e_stat_sz0,weights,operator_circuits):.8f}")
    print(f" The first excited state Energy = {E3:.8f} Ha, Sz = {Sz_fn(e_stat_szm1,weights,operator_circuits):.8f}")
    print("Number of gates = {}\n".format(len(operator_circuits)))
    configs=qml.specs(cost_h_fn)(g_stat,weights, excitations=operator_circuits)
    print("Circuit depth {}\n".format(configs['resources'].depth))

    if np.abs(E0-E0_pev)<cov and np.abs(E1-E1_pev)<cov and np.abs(E3-E3_pev)<cov:
        break

    E0_pev=E0
    E1_pev=E1
    E2_pev=E2
    E3_pev=E3


Epoch = 0, G Energy = -7.60898166 Ha, E1 Energy = -7.49074895 Ha,
 E2 Energy = -7.49074895 Ha
Number of gates = 0

Circuit depth 1

Max grad 0.12296176040542098
Epoch = 1, G Energy = -7.61431460 Ha, Sz = 0.00000000 Ha,
 The first excited state Energy = -7.49074895 Ha, Sz= 1.00000000
 The first excited state Energy = -7.49074895 Ha, Sz= 0.00000000
 The first excited state Energy = -7.49074895 Ha, Sz = -1.00000000
Number of gates = 1

Circuit depth 2

Max grad 0.09384808786854848
Epoch = 2, G Energy = -7.61967928 Ha, Sz = 0.00000000 Ha,
 The first excited state Energy = -7.49074895 Ha, Sz= 1.00000000
 The first excited state Energy = -7.49074895 Ha, Sz= 0.00000000
 The first excited state Energy = -7.49074895 Ha, Sz = -1.00000000
Number of gates = 2

Circuit depth 3

Max grad 0.0734423026356488
Epoch = 3, G Energy = -7.62102202 Ha, Sz = 0.00000000 Ha,
 The first excited state Energy = -7.49074895 Ha, Sz= 1.00000000
 The first excited state Energy = -7.49263893 Ha, Sz= 0.00000000
 The fir